In [ ]:
import requests

In [ ]:
# Query for some customers and their purchase history
resp = requests.get(
    url=("http://server:3000/"
         "customer?select=first_name,last_name,"
         "purchases:purchase(transaction_date,count,item_cost,inventory(item_name))&"
         "limit=5")
)

for entry in resp.json():
    print(f"{entry['first_name']} {entry['last_name']} - purchase history")
    for purchase in entry['purchases']:
        dt = purchase['transaction_date']
        n_item, item_cost = purchase['count'], purchase['item_cost']
        total_cost = n_item * item_cost
        item_name = purchase['inventory']['item_name']
        print(f" - {dt}: {n_item} {item_name} at ${item_cost:.02f} each, ${total_cost:.02f} total")
    print()

In [ ]:
# Only show purchases where the item cost $600 or more
resp = requests.get(
    url=("http://server:3000/"
         "customer?select=first_name,last_name,"
         "purchases:purchase(transaction_date,count,item_cost,inventory(item_name))&"
         "purchase.item_cost=gte.600&"
         "limit=5")
)

for entry in resp.json():
    print(f"{entry['first_name']} {entry['last_name']} - purchase history")
    for purchase in entry['purchases']:
        dt = purchase['transaction_date']
        n_item, item_cost = purchase['count'], purchase['item_cost']
        total_cost = n_item * item_cost
        item_name = purchase['inventory']['item_name']
        print(f" - {dt}: {n_item} {item_name} at ${item_cost:.02f} each, ${total_cost:.02f} total")
    print()

In [ ]:
# Total sales for January 2019
resp = requests.get(
    url=("http://server:3000/"
         "purchase?select=count,item_cost,transaction_date&"
         "and=(transaction_date.gte.'2019-01-01',"
         "transaction_date.lt.'2019-02-01')")
)
assert len({
    entry['transaction_date']
    for entry in resp.json()
}) <= 31

total = sum([e['count']*e['item_cost'] for e in resp.json()])
print(f"Total sales for Jan-2019: ${total:,.2f}")

In [ ]:
# Total sales by store, for January 2019
resp = requests.get(
    url=("http://server:3000/"
         "purchase?select=count,item_cost,transaction_date,store(name)&"
         "and=(transaction_date.gte.'2019-01-01',"
         "transaction_date.lt.'2019-02-01')")
)
per_store = dict()
for entry in resp.json():
    store = entry['store']['name']
    per_store[store] = per_store.setdefault(store, 0) + (entry['count'] * entry['item_cost'])
for store, total in per_store.items():
    print(f"Total sales, Jan-2019, for {store}: ${total:,.2f}")

In [ ]:
def check_resp(resp, op, status_code=200, json_len=1, local_json={}):
    assert resp.status_code == status_code, f"{op} failed: {resp.status_code}"
    if status_code == 200:
        resp_json = resp.json()
        assert isinstance(resp_json, list)
        l = len(resp.json())
        assert l == json_len, f"invalid response length {l}, should be {json_len}"
        for k, v in local_json.items():
            assert resp_json[0][k] == v, "json response not as expected"
    print(f"{op} succeeded")

user_definition = {
    "uid": 1000,
    "first_name": "Robert",
    "last_name": "Paulson",
    "dob": "1999-11-11",
    "phone_number": '12345678',
    "street_address": "420 Paper St. Wilmington DE 19886",
    "email": "first_rule@fight.club.is"
}

# Test all CRUD operations
check_resp(
    requests.post(url="http://server:3000/customer", json=user_definition), 
    "create", 201
)
check_resp(
    requests.get(url="http://server:3000/customer?uid=eq.1000"), 
    "retrieve", local_json=user_definition
)
user_definition['phone_number'] = '87654321'
check_resp(
    requests.put(url="http://server:3000/customer?uid=eq.1000", json=user_definition),
    "update", status_code=204
)
check_resp(
    requests.get(url="http://server:3000/customer?uid=eq.1000"), 
    "re-retrieve", local_json=user_definition
)
check_resp(
    requests.delete("http://server:3000/customer?uid=eq.1000"), "delete", 
    status_code=204
)
check_resp(
    requests.get(url="http://server:3000/customer?uid=eq.1000"), 
    "confirm not present", json_len=0
)